# Exercício 1 - grupo 6 - Ana Margarida Campos (A85166) , Nuno Pereira (PG42846)

Neste primeiro exercício foi proposta a implementação de duas versões, IND-CPA e IND-CCA, do protótipo **NTRU** que foi candidato ao concurso *NIST-PQC*, neste caso da terceira ronda.
<br/>
De seguida, são apresentadas em duas seções (Passively secure DPKE e Strongly secure KEM) os resultados da resolução deste exercício acompanhado de uma explicação detalhada de cada função implementada. Este, foi desenvolvido tendo por base o documento *ntru.pdf*. 

## Passively secure DPKE

A primeira implementação consistiu no desenvolvimento de funções que permitissem uma segurança IND-CPA, ou seja, segurança contra ataques *Chosen Plaintext Attacks*.
<br/>
Numa primeira fase foi necessário implementar algumas funções auxiliares, sendo estas:

- ***\_ternary***: cria uma lista com elementos entre -1 a 1 e, posteriormente, com base nesta lista, retorna um polinómio ternário (ou seja com coeficientes -1,0 e 1);

- ***\_fixedType***: cria uma lista com elementos entre -1 a 1 mas com a particularidade de existir Q/16-1 elementos iguais a 1 e Q/16-1 elementos iguais a -1. Esta lista é depois convertida em polinómio ternário;

- ***sample\_fg***: faz recurso de ambas funções anteriores (***\_ternary*** e ***\_fixedType***), obtendo, deste modo, os polinómios necessários para a obtenção das chaves;

- ***pack***: recebe como argumento um polinómio e codifica-o em *bytes*;

- ***unpack***: contrário da função ***pack***, ou seja, recebe um conjunto de *bytes* e retorna um polinómio do tipo *Rq*;

- ***unpackSq***: recebe um conjunto de *bytes* e retorna um polinómio do tipo *Sq*;

- ***chave\_publica***: utiliza os polinómios criados com recurso à função ***sample\_fg*** e retorna um *h* e *hq* que vão ser fundamentais para a criação das chaves pública e privada.

As funções principais centram-se na geração de chaves, cifragem da mensagem passada como parâmetro e posterior decifragem do texto cifrado, obtendo deste modo, a mensagem original:

- ***gerar\_chaves***: com recurso às funções anteriormente apresentadas, ocorre a geração das chaves pública e privada. A primeira é essencial para a cifragem da mensagem e a segunda para a decifragem do criptograma;

- ***cifragem***: esta função tem como objetivo principal cifrar uma mensagem. Desta forma, recebe como parâmetros a chave pública e a mensagem e, com recurso às funções anteriores, cria o texto cifrado;

- ***decifragem***: tem como objetivo decifrar um criptograma, obtendo como resultado o texto limpo correspondente. Recebe como argumentos a chave privada e o criptograma.

Uma dificuldade encontrada centrou-se na obtenção do texto limpo correto, pelo que, a maneira utilizada para ultrapassar este obstáculo foi a da cifragem do parâmetro *m1* utilizando o modo *GCM*, e a posterior decifragem, permitindo, deste modo, obter o resultado esperado.

In [1]:
# imports necessários para a resolução
import random as rn
import numpy as np
from sympy import Symbol, Poly
import os
import math
import zlib 
import gzip
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

In [2]:
# constantes do NTRU
N = 509
Q = 2048
T = N//4

# criação dos anéis necessários
_Z.<w>  = ZZ[]
R.<w>   = QuotientRing(_Z ,_Z.ideal(w^N - 1))

_Q.<w>  = GF(Q)[]
Rq.<w>  = QuotientRing(_Q , _Q.ideal(w^N - 1))

_Q.<w>  = GF(Q)[]
RT.<w>  = QuotientRing(_Q , _Q.ideal(w^204))

_E.<w>  = ZZ[]
S.<w>   = QuotientRing(_E,_E.ideal(w^N - 1))

_Q.<w>  = GF(Q)[]
Sq.<w>  = QuotientRing(_Q , _Q.ideal((w^N - 1)/(w-1)))

_Q3.<w>  = GF(3)[]
S3.<w>  = QuotientRing(_Q3 , _Q3.ideal((w^N - 1)/(w-1)))


# funções auxiliares:
metadados = os.urandom(16)
listanouce = []
# gerador de nounce
def geraNounce(tamNounce):
    nounce = os.urandom(tamNounce)
    if not (nounce in listanouce):
        listanouce.append(nounce)
        return nounce
    else:
        geraNounce(tamNounce)

# criação de um polinómio ternário
def _ternary(n=N,t=T):
    u = [rn.choice([-1,1]) for i in range(t)] + [0]*(8*(n-1)-t)
    rn.shuffle(u)
    return Rq(u)

# criação de um polinómio ternário com igual número de elementos iguais a -1 e 1
def _fixedType(n=N,t=T):
    q = Q//16 -1
    h = (30*(n-1))-2*q
    u1 = [rn.choice([1]) for i in range(t)] + [0]*(q-t)
    u2 = [rn.choice([-1]) for i in range(t)] + [0]*(q-t)
    u3 = [rn.choice([0]) for i in range(t)] + [0]*(h-t)
    u = [*u1,*u2,*u3]
    rn.shuffle(u)
    return Rq(u)

# criação dos polinómios com recurso às funções anteriores
def sample_fg(n=N, t=T):
    f = _ternary(n,t)
    g = _fixedType(n,t)
    return f,g

# obtenção do tamanho necessário para o unpack
def tamanho(stringB, numberS):
    count = 2
    auxCount = 1
    i = 0
    while i < len(stringB):
        if numberS == auxCount:
            i = i + 2
            while (i < len(stringB)) and (stringB[i] != 120 or stringB[i + 1] != 1):
                count = count + 1
                i = i + 1
            auxCount = auxCount + 1

        i = i + 1
        if (i + 2) < len(stringB) and (stringB[i] == 120 and stringB[i + 1] == 1):
            auxCount = auxCount + 1
        if auxCount > numberS:
            break
    return count

# passagem do polinómio para bytes
def pack(polinomio):
    check_List=isinstance(polinomio, list)
    if(not check_List):
        polinomio=polinomio.list()
        polinomioB= bytes(_Z(polinomio))
        compress = zlib.compress(polinomioB,1)
    else:
        compress = zlib.compress(bytes(_Z(polinomio)),1)
    return compress

# passagem de um conjunto de bytes para um polinómio do tipo Rq
def unpack(pack):
    unpack = zlib.decompress(pack)
    newUnpack=[]
    for i in unpack:
        newUnpack.append(i)
    return Rq(newUnpack)

# passagem de um conjunto de bytes para um polinómio do tipo Sq
def unpackSq(pack):
    unpack = zlib.decompress(pack)
    newUnpack=[]
    for i in unpack:
        newUnpack.append(i)
    return Sq(newUnpack)

# geraçãp de um h e hq que vão ser necessários na criação das chaves públicas e privadas
def chave_publica(f, g):
    G = g * 3
    v0 = Sq(G*f)
    v1 = v0.inverse_of_unit()
    h = Rq(v1*G*G)
    hq = Rq(v1*f*f)
    return(h,hq)

# funções principais:

# geração das chaves pública e privada
def gerar_chaves():
    f, g = sample_fg()
    fq = f.inverse_of_unit()
    h, hq = chave_publica(f,g)
    pf= pack(f)
    pfq =pack(fq)
    phq =pack(hq)
    packed_private_key = pf+ pfq+phq
    packed_public_key = pack(h)
    return (packed_private_key, packed_public_key)

# cifragem de uma mensagem
def cifragem(packed_public_key, packed_rm, key):
    packed_r = pack(packed_rm[:102])
    packed_m = pack(packed_rm[-102:])
    r = unpack(packed_r)
    m0 = unpack(packed_m)
    m1 = m0.lift()
    h = unpack(packed_public_key)
    c = Rq(r*h + m1)
    packed_ciphertext = pack(c)
    aesgcm = AESGCM(key)
    nonce = geraNounce(12)
    m1_cifrado = aesgcm.encrypt(nonce, bytes(_Z(m1)), metadados)
    m1_cifrado += nonce
    return packed_ciphertext, m1_cifrado

# decifragem de um criptograma obtendo a mensagem original
def decifragem(packed_private_key, packed_ciphertext, key, m1_cifrado):
    tf = tamanho(packed_private_key,1)
    tfq = tamanho(packed_private_key,2)
    thq = tamanho(packed_private_key,3)
    packed_f =  packed_private_key[:tf]
    packed_private_key = packed_private_key[tf:]
    packed_fq = packed_private_key[:tfq]
    packed_hq = packed_private_key[tfq:]
    c = Rq(unpack(packed_ciphertext))
    f = Rq(unpack(packed_f))
    fq = unpack(packed_fq)
    hq = unpackSq(packed_hq)
    aesgcm = AESGCM(key)
    nonce = m1_cifrado[-12:]
    m1_cifrado = m1_cifrado[:-12]
    m1 = aesgcm.decrypt(nonce, m1_cifrado, metadados)
    y = []
    for i in m1:
        y.append(i)
    m1_novo = Rq(y).lift()
    r = Rq((c-m1_novo)*hq)
    packed_rm = [*r, *(m1_novo.list())]
    fail = 0
    for i in r.list():
        if i==1 or i==0 or i==-1:
            fail = 0
        else:
            fail = 1
            break
    for i in m1_novo.list():
        if i==1 or i==0 or i==-1:
            fail = 0
        else:
            fail = 1
            break
    result = packed_rm[:102] + packed_rm[-102:]
    return result, fail
        

De seguida são apresentados os resultados obtidos com recurso às funções anteriores. Neste caso, estamos a considerar uma mensagem fixa.

In [3]:
key = os.urandom(32)

mensagem = RT([1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
            0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
            1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
            1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
            1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1])

print("Mensagem=", mensagem,"\n")

packed_private_key, packed_public_key = gerar_chaves()
print("Chave privada=", packed_private_key, "\n")
print("Chave pública=", packed_public_key, "\n")

texto_cifrado, m1_cifrado= cifragem(packed_public_key, mensagem.list(), key)
print("texto cifrado=", texto_cifrado,"\n")


texto_limpo,fail = decifragem(packed_private_key, texto_cifrado,key, m1_cifrado)
print("decifragem polinomio==", RT(texto_limpo),"\n")

print("Texto Limpo = Resultado da decifragem:", texto_limpo== mensagem.list())



Mensagem= w^203 + w^202 + w^201 + w^199 + w^198 + w^197 + w^195 + w^193 + w^191 + w^190 + w^189 + w^188 + w^187 + w^184 + w^183 + w^182 + w^181 + w^180 + w^179 + w^178 + w^177 + w^176 + w^175 + w^174 + w^173 + w^172 + w^170 + w^169 + w^167 + w^166 + w^164 + w^163 + w^160 + w^156 + w^154 + w^153 + w^152 + w^151 + w^149 + w^148 + w^147 + w^146 + w^145 + w^144 + w^143 + w^141 + w^139 + w^137 + w^135 + w^134 + w^132 + w^131 + w^130 + w^129 + w^128 + w^127 + w^126 + w^125 + w^124 + w^123 + w^122 + w^120 + w^119 + w^118 + w^117 + w^116 + w^115 + w^114 + w^113 + w^112 + w^111 + w^110 + w^108 + w^106 + w^105 + w^103 + w^102 + w^100 + w^99 + w^97 + w^96 + w^95 + w^94 + w^93 + w^92 + w^90 + w^89 + w^88 + w^86 + w^84 + w^82 + w^81 + w^79 + w^77 + w^75 + w^74 + w^73 + w^72 + w^71 + w^70 + w^67 + w^64 + w^63 + w^61 + w^60 + w^59 + w^58 + w^57 + w^56 + w^55 + w^54 + w^52 + w^49 + w^47 + w^46 + w^44 + w^43 + w^42 + w^41 + w^40 + w^38 + w^35 + w^34 + w^33 + w^32 + w^31 + w^30 + w^29 + w^26 + w^25 + w^

## Strongly secure KEM

A segunda implementação consistiu no desenvolvimento de funções que permitissem uma segurança IND-CCA, ou seja, segurança contra ataques *Chosen Ciphertext Attacks*.
<br/>
Tal como anteriormente, tornou-se necessário, numa primeira fase, a implementação de funções auxiliares. Estas são iguais às desenvolvidas previamente, com algumas diferenças mínimas, nomeadamente nas funções de ***pack*** e ***unpack***. Foram também implementadas duas funções auxiliares novas sendo estas:

- ***bytes\_to\_bits***: tranforma um conjunto de *bytes* em *bits*;

- ***bits\_to\_bytes***: transforma um conjunto de *bits* em *bytes*;

- ***rand\_bits***: gera palavras de *bits* com tamanho passado como argumento.

As funções principais são agora a geração de chaves, o encapsulamento e o desencapsulamento:

- ***geracao\_chaves***: com recurso às funções auxiliares, ocorre a geração das chaves pública e privada;

- ***encapsulate***: recebe como argumento a chave pública e retorna a chave partilhada bem como o *packed_ciphertext* que corresponde à cifragem do *packed\_rm*;

- ***decapsulate***: tem como objetivo a obtenção da chave partilhada. Para tal recebe como argumentos a chave privada e o criptograma e, através de um conjunto de operações, devolve o esperado, a chave partilhada.
    

In [4]:
# funções auxiliares:

# obtenção do tamanho necessário para o unpack
def tamanho(stringB, numberS):
    count = 10
    auxCount = 1
    i = 0
    while i < len(stringB):
        if numberS == auxCount:
            i = i + 10
            while (i < len(stringB)) and (stringB[i] != 31 or stringB[i + 1] != 139 or stringB[i + 2] != 8 or stringB[i + 3] != 0  ):
                count = count + 1
                i = i + 1
            auxCount = auxCount + 1

        i = i + 1
        if (i + 10) < len(stringB) and (stringB[i] == 31 and stringB[i + 1] == 139 and stringB[i + 2] == 8 and stringB[i + 3] == 0 ):
            auxCount = auxCount + 1
        if auxCount > numberS:
            break
    return count

# geração das chaves pública e privada
def gerar_chaves():
    f, g = sample_fg()
    fq = f.inverse_of_unit()
    h, hq = chave_publica(f,g)
    pf= pack2(f)
    pfq =pack2(fq)
    phq =pack2(hq)
    packed_private_key = pf+ pfq+phq
    packed_public_key = pack2(h)
    return (packed_private_key, packed_public_key)

# gera palavras de *bits* com tamanho passado como argumento
def rand_bits(p):
    key1 = ""
    for i in range(p):
        temp = str(rn.randint(0, 1))
        key1 += temp     
    return(key1)

# conversão de um conjunto de bytes para bits
def bytes_to_bits(s):
    s = s.decode('ISO-8859–1')
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    u=""
    for i in result:
        u = u + str(i)
    return u

# conversão de um conjunto de bits para bytes
def bits_to_bytes(bits):
    chars = []
    for b in range(int(len(bits) / 8)):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars).encode('ISO-8859–1')

# passagem de um polinomio para um conjunto de bytes
def pack2(polinomio):
    check_List=isinstance(polinomio, list)
    if(not check_List):
        polinomio=polinomio.list()
        polinomioB= bytes(_Z(polinomio))
        compress = gzip.compress(polinomioB)
    else:
        polinomioB= bytes(_Z(polinomio))
        compress = gzip.compress(polinomioB)
    return compress

# passagem de bytes para polinómio Rq
def unpack(compress):
    unpack = gzip.decompress(compress)
    newUnpack=[]
    for i in unpack:
        newUnpack.append(i)
    return Rq(newUnpack)

# passagem de bytes para polinómio Sq
def unpackSq(compress):
    unpack = gzip.decompress(compress)
    newUnpack=[]
    for i in unpack:
        newUnpack.append(i)
    return Sq(newUnpack)

# cifragem igual à anterior com modificação do pack
def cifragem2(packed_public_key, packed_rm, key):
    lista = []
    for i in packed_rm:
        lista.append(i)
    packed_r = packed_rm[:(tamanho(lista,1))]
    packed_m = packed_rm[-(tamanho(lista,2)):]
    r = unpack(packed_r)
    m0 = unpack(packed_m)
    m1 = m0.lift()
    h = unpack(packed_public_key)
    c = Rq(r*h + m1)
    packed_ciphertext = pack2(c)
    aesgcm = AESGCM(key)
    nonce = geraNounce(12)
    m1_cifrado = aesgcm.encrypt(nonce, bytes(_Z(m1)), metadados)
    m1_cifrado += nonce
    return packed_ciphertext, m1_cifrado

# decifragem igual à anterior com modificação do pack
def decifragem2(packed_private_key, packed_ciphertext, key, m1_cifrado):
    tf = tamanho(packed_private_key,1)
    tfq = tamanho(packed_private_key,2)
    thq = tamanho(packed_private_key,3)
    packed_f =  packed_private_key[:tf]
    packed_private_key = packed_private_key[tf:]
    packed_fq = packed_private_key[:tfq]
    packed_hq = packed_private_key[tfq:]
    c = Rq(unpack(packed_ciphertext))
    f = Rq(unpack(packed_f))
    fq = unpack(packed_fq)
    hq = unpackSq(packed_hq)
    aesgcm = AESGCM(key)
    nonce = m1_cifrado[-12:]
    m1_cifrado = m1_cifrado[:-12]
    m1 = aesgcm.decrypt(nonce, m1_cifrado, metadados)
    y = []
    for i in m1:
        y.append(i)
    m1_novo = Rq(y).lift()
    r = Rq((c-m1_novo)*hq)
    for i in r.list():
        if i==1 or i==0 or i==-1:
            fail = 0
        else:
            fail = 1
            break
    for i in m1_novo.list():
        if i==1 or i==0 or i==-1:
            fail = 0
        else:
            fail = 1
            break
    packed_rm = pack2(r) + pack2(m1_novo.list())
    return packed_rm,fail

# funções principais:


# gera as chaves pública e privada
def geracao_chaves():
    fg_bits = rand_bits(19304)
    prf_key = rand_bits(256)
    packed_dpke_private_key, packed_public_key = gerar_chaves()
    packed_private_key = packed_dpke_private_key + bits_to_bytes(prf_key)
    return packed_private_key, packed_public_key

# encapsulamento do packed_rm
def encapsulate(packed_public_key, key):
    coins = rand_bits(19304)
    r, m = sample_fg()
    packed_rm = pack2(r) + pack2(m)
    shared_key = hash(bytes_to_bits(packed_rm))
    packed_ciphertext, m1_cifrado = cifragem2(packed_public_key, packed_rm,key)
    return shared_key, packed_ciphertext, m1_cifrado
    
# desancapsulamento obtendo a chave partilhada
def decapsulate(packed_private_key, packed_ciphertext, key, m1_cifrado):
    prf_key = packed_private_key[-32:]
    packed_dpke_private_key = packed_private_key[:(len(packed_private_key)-len(prf_key))]
    tf = tamanho(packed_dpke_private_key,1)
    tfq = tamanho(packed_dpke_private_key,2)
    thq = tamanho(packed_dpke_private_key,3)
    packed_f =  packed_dpke_private_key[:tf]
    packed_dp = packed_dpke_private_key[tf:]
    packed_fq = packed_dp[:tfq]
    packed_hq = packed_dp[tfq:]
    packed_rm, fail = decifragem2(packed_dpke_private_key,packed_ciphertext,key, m1_cifrado)
    shared_key = hash(bytes_to_bits(packed_rm))
    concat = bytes_to_bits(prf_key) + bytes_to_bits(packed_ciphertext)
    random_key = hash(concat)
    if fail == 0:
        return shared_key
    else:
        return random_key

De seguida são apresentados os resultados obtidos com recurso às funções anteriores.

In [5]:
key = os.urandom(32)
packed_private_key, packed_public_key = geracao_chaves()
print("Chave pública=", packed_public_key, "\n")
print("Chave privada=", packed_private_key, "\n")
shared_key, packed_ciphertext, m1_cifrado = encapsulate(packed_public_key, key)
print("Shared key cifragem =", shared_key, "\n")
shared_key2 = decapsulate(packed_private_key, packed_ciphertext,key, m1_cifrado)
print("Shared key decifragem=", shared_key2, "\n")
print("Shared key cifragem = Shared key decifragem", shared_key==shared_key2)



Chave pública= b'\x1f\x8b\x08\x00\x94\xfa\x9b`\x02\xffUQ\x01\x12\xc0 \x08\x82\xff\x7f\xba\xb3\x00\xa9\xb5\xb3T\x10\r\x00\x89Y\x9c\x05\xde\xc3\x18\xbc\xcb|\xd7\xeaW\xf0\x1e\xe0\xfcGa\x1e\x04\xbf\xd9X\xa7\x01\xfc(\xb6\xd8\xbb0;\n\xcd\xb8D\x92&\x04\x16*U,\x04\x95g}i\x93V\x1dY\xe8m\xf1\xc2\xda\x9b6\x15\x8b*\x974\x0c\xd5+7\x9ez=B9Y\x03I\xc5\xcc\x98y\x1c|\xc45P\r\x8e\xff\x9b\xb6\xf8\xb1\x07\x92\xd9\xef\xaa\xfc\x01\x00\x00' 

Chave privada= b'\x1f\x8b\x08\x00\x94\xfa\x9b`\x02\xff\x95\x91Q\x0e\xc0 \x08Cy\xf7\xbf\xf4\x96!\xac\xd0\x9f\xcdD\xd1\x12J\x8bA\x10\xf7Z\x87\xc4\xbc\xd3\x00\r\xe4\x83\'\x05^\xb5\x88F\xe1\x9f\x05\xa7\x0c\x17\x95\x9b\nGR\x94$\xa6(6`"E.\xa3\xbd\x8e\xa0\x9a\x18\xfb\x9b\xa39>\xf8\xad)\x82\xe3\xa1\xde\xed\x83\xd26\xa1\xee\x85\x87\xad\xac\x06u\x01Y\x05cj\xf7\x01\x00\x00\x1f\x8b\x08\x00\x94\xfa\x9b`\x02\xff]P\x8b\x12\xc0 \x08\x82\xff\xff\xe9m\n\x8a\xeb\xeaJSy\x80\xa8\xf5^\xec\xc3Z_\x86\x9d\xae\x0f\xd0U]\xa3\xb8\x1e\x84;;\xa0\xd2\xd4\xd9F\r\x89hrpu\xa3\x9e[=H\x94\xc4\xd4\x182\xa0M